In [1]:
import pandas as pd

In [5]:
# Read & join data
organic_data = pd.read_excel("conjoint_survey_organic.xlsx")
ads_data = pd.read_csv("conjoint_survey_ads.csv")

data = pd.concat([organic_data, ads_data], ignore_index=True)
data.head()

,Timestamp,"Berapa nomer telepon anda? Nomer ini akan digunakan untuk membagikan GoPay Rp 50.000 per orang, hasil undian untuk 100 orang. Kami hanya akan mengirimkan ke pengisi kuisioner yang valid, i.e. jawaban tidak random.",1. Produk manakah yang akan anda beli? (Anda bisa memilih membeli (klik) lebih dari 1 pilihan),2. Produk manakah yang akan anda beli? (Anda bisa memilih membeli (klik) lebih dari 1 pilihan),3. Produk manakah yang akan anda beli? (Anda bisa memilih membeli (klik) lebih dari 1 pilihan),4. Produk manakah yang akan anda beli? (Anda bisa memilih membeli (klik) lebih dari 1 pilihan),5. Produk manakah yang akan anda beli? (Anda bisa memilih membeli (klik) lebih dari 1 pilihan),6. Produk manakah yang akan anda beli? (Anda bisa memilih membeli (klik) lebih dari 1 pilihan),7. Produk manakah yang akan anda beli? (Anda bisa memilih membeli (klik) lebih dari 1 pilihan),8. Produk manakah yang akan anda beli? (Anda bisa memilih membeli (klik) lebih dari 1 pilihan),9. Produk manakah yang akan anda beli? (Anda bisa memilih membeli (klik) lebih dari 1 pilihan),10. Produk manakah yang akan anda beli? (Anda bisa memilih membeli (klik) lebih dari 1 pilihan)
0,2023-03-08 18:47:23,08xx336019xxx,B,A,A,A,A,B,A,C,A,A
1,2023-03-08 18:48:27,08xx10001xxx,C,"C, D. Tidak memilih semua product",C,C,"A, B","A, B",A,A,A,A
2,2023-03-08 20:17:52,08xx56592xxx,D. Tidak memilih semua product,D. Tidak memilih semua product,"A, C",A,A,C,D. Tidak memilih semua product,D. Tidak memilih semua product,D. Tidak memilih semua product,A
3,2023-03-13 20:59:49,08xx391631xxx,D. Tidak memilih semua product,D. Tidak memilih semua product,"A, C",A,A,C,D. Tidak memilih semua product,D. Tidak memilih semua product,D. Tidak memilih semua product,A
4,2023-03-13 21:01:45,08xx21820xxx,D. Tidak memilih semua product,D. Tidak memilih semua product,C,A,A,D. Tidak memilih semua product,D. Tidak memilih semua product,D. Tidak memilih semua product,D. Tidak memilih semua product,D. Tidak memilih semua product


In [7]:
# Change columns name 
data.columns = ["Timestamp", "Phone", "Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7", "Q8", "Q9", "Q10"]

# DATA CLEANING
# 1. Check for duplicates across all columns
duplicates = data[data.duplicated()]
print("1. Check for duplicates")

if not duplicates.empty:
    print("Duplicate rows found:")
    print(duplicates)
else:
    print("No duplicate rows found.")

# 2. Check data format
print("2. Check data format")
types = data.dtypes
print("Data format in each column:")
print(types)


1. Check for duplicates
No duplicate rows found.
2. Check data format
Data format in each column:
Timestamp    object
Phone        object
Q1           object
Q2           object
Q3           object
Q4           object
Q5           object
Q6           object
Q7           object
Q8           object
Q9           object
Q10          object
dtype: object


In [8]:
# 3. Define a function to filter and keep valid answers
# Valid answer is when there is only 1 answer, also when there are 2 answers without 'D. Tidak memilih semua produk'

def filter_valid_answers(row):
    '''
    Function to filter and keep valid answers, that defined as:
    - For each column (starting from the third column), if there are exactly 2 answers
      and neither of them is "D. Tidak memilih semua product," they are considered valid.
    - If there is only 1 answer in a column, it is also considered valid.
    
    Parameters:
        row : A row of data containing responses to survey questions.

    Returns:
        valid_answers :  Dictionary containing phone number and valid answers for each column
    '''
    valid_answers = {} # Create blank list for valid answer
    valid_answers["Phone"] = row["Phone"]
    for col in data.columns[2:]:  # Skip the first two columns (Timestamp & Phone)
        answers = row[col].split(", ")
        
        if len(answers) == 2 and "D. Tidak memilih semua product" not in answers: # valid: there are 2 answers and none of them is "D"
            valid_answers[col] = ", ".join(answers)
        elif len(answers) == 1: # valid: there is only 1 answer
            valid_answers[col] = ", ".join(answers)

    return valid_answers

# Apply the filtering function to each row
filtered_data = data.apply(filter_valid_answers, axis=1)
filtered_data = filtered_data.reset_index(drop=True)
filtered_data = pd.DataFrame(list(filtered_data))
filtered_data.head()


,Phone,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,08xx336019xxx,B,A,A,A,A,B,A,C,A,A
1,08xx10001xxx,C,NaN,C,C,"A, B","A, B",A,A,A,A
2,08xx56592xxx,D. Tidak memilih semua product,D. Tidak memilih semua product,"A, C",A,A,C,D. Tidak memilih semua product,D. Tidak memilih semua product,D. Tidak memilih semua product,A
3,08xx391631xxx,D. Tidak memilih semua product,D. Tidak memilih semua product,"A, C",A,A,C,D. Tidak memilih semua product,D. Tidak memilih semua product,D. Tidak memilih semua product,A
4,08xx21820xxx,D. Tidak memilih semua product,D. Tidak memilih semua product,C,A,A,D. Tidak memilih semua product,D. Tidak memilih semua product,D. Tidak memilih semua product,D. Tidak memilih semua product,D. Tidak memilih semua product


In [9]:
# Invalid data (NaN) is considered as not choosing an answer, so invalid data can be interpreted as choosing D. 
# Change NaN to 'D. Tidak memilih semua produk'

filtered_data.fillna('D. Tidak memilih semua product', inplace=True)
filtered_data.head()

,Phone,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,08xx336019xxx,B,A,A,A,A,B,A,C,A,A
1,08xx10001xxx,C,D. Tidak memilih semua product,C,C,"A, B","A, B",A,A,A,A
2,08xx56592xxx,D. Tidak memilih semua product,D. Tidak memilih semua product,"A, C",A,A,C,D. Tidak memilih semua product,D. Tidak memilih semua product,D. Tidak memilih semua product,A
3,08xx391631xxx,D. Tidak memilih semua product,D. Tidak memilih semua product,"A, C",A,A,C,D. Tidak memilih semua product,D. Tidak memilih semua product,D. Tidak memilih semua product,A
4,08xx21820xxx,D. Tidak memilih semua product,D. Tidak memilih semua product,C,A,A,D. Tidak memilih semua product,D. Tidak memilih semua product,D. Tidak memilih semua product,D. Tidak memilih semua product,D. Tidak memilih semua product


In [10]:
# Check how much user in data
total_rows = len(filtered_data)
print("Total user:", total_rows)

Total user: 285


In [11]:
# DATA MANIPULATION

# Create dictionary for each answer in Q1-Q10 
options = {
    "Q1": {
        "A": {"Skill":"Create Analytics Dashboard", "Program":"Tutorial Based", "Price":"Rp 500.000,0"},
        "B": {"Skill":"Perform Customer Segmentation","Program": "Mentoring Based", "Price":"Rp 350.000,0"},
        "C": {"Skill":"Design AB Test Experimentation", "Program":"Mentoring Based", "Price":"Rp 300.000,0"}
    },
    "Q2": {
        "A": {"Skill":"Create Analytics Dashboard", "Program":"Tutorial Based", "Price": "Rp 500.000,0"},
        "B": {"Skill":"Design Data Pipeline", "Program": "Mentoring Based", "Price": "Rp 300.000,0"},
        "C": {"Skill":"Perform Credit Scoring Analytics", "Program": "Mentoring Based", "Price": "Rp 550.000,0"}
    },
    "Q3": {
        "A": {"Skill":"Perform Customer Segmentation", "Program": "Mentoring Based", "Price": "Rp 350.000,0"},
        "B": {"Skill":"Perform Customer Segmentation", "Program": "Tutorial Based", "Price": "Rp 450.000,0"},
        "C": {"Skill":"Design Data Pipeline", "Program":"Mentoring Based", "Price": "Rp 250.000,0"}
    },
    "Q4": {
        "A": {"Skill": "Design AB Test Experimentation", "Program": "Mentoring Based", "Price": "Rp 500.000,0"},
        "B": {"Skill": "Perform Price Optimization", "Program": "Tutorial Based", "Price": "Rp 350.000,0"},
        "C": {"Skill": "Perform Credit Scoring Analytics", "Program": "Mentoring Based", "Price": "Rp 350.000,0"}
    },
    "Q5": {
        "A": {"Skill": "Design Data Pipeline", "Program": "Mentoring Based", "Price": "Rp 400.000,0"},
        "B": {"Skill": "Perform Customer Lifetime Analysis", "Program": "Tutorial Based", "Price": "Rp 300.000,0"},
        "C": {"Skill": "Design AB Test Experimentation", "Program": "Tutorial Based", "Price": "Rp 300.000,0"}
    },
    "Q6": {
        "A": {"Skill": "Perform Churn Analysis", "Program": "Tutorial Based", "Price": "Rp 450.000,0"},
        "B": {"Skill": "Perform Customer Segmentation", "Program": "Mentoring Based", "Price": "Rp 300.000,0"},
        "C": {"Skill": "Create Machine Learning Model", "Program": "Mentoring Based", "Price": "Rp 300.000,0"}
    },
    "Q7": {
        "A": {"Skill": "Perform Customer Lifetime Analysis", "Program": "Tutorial Based", "Price": "Rp 500.000,0"},
        "B": {"Skill": "Design Data Pipeline", "Program": "Mentoring Based", "Price": "Rp 550.000,0"},
        "C": {"Skill": "Deploy Machine Learning Model", "Program": "Tutorial Based", "Price": "Rp 350.000,0"}
    },
    "Q8": {
        "A": {"Skill": "Perform Credit Scoring Analysis", "Program": "Mentoring Based", "Price": "Rp 300.000,0"},
        "B": {"Skill": "Design Data Pipeline", "Program": "Mentoring Based", "Price": "Rp 550.000,0"},
        "C": {"Skill": "Create Machine Learning Model", "Program": "Tutorial Based", "Price": "Rp 550.000,0"}
    },
    "Q9": {
        "A": {"Skill": "Create Analytics Dashboard", "Program": "Mentoring Based", "Price": "Rp 250.000,0"},
        "B": {"Skill": "Design AB Test Experimentation", "Program": "Tutorial Based", "Price": "Rp 550.000,0"},
        "C": {"Skill": "Perform Customer Lifetime Analysis", "Program": "Mentoring Based", "Price": "Rp 350.000,0"}
    },
    "Q10": {
        "A": {"Skill": "Perform Credit Scoring Analysis", "Program": "Mentoring Based", "Price": "Rp 400.000,0"},
        "B": {"Skill": "Perform Churn Analysis", "Program": "Mentoring Based", "Price": "Rp 450.000,0"},
        "C": {"Skill": "Perform Churn Analysis", "Program": "Tutorial Based", "Price": "Rp 500.000,0"}
       
    }
}

# Define a function to process each row
def process_row(row):
    '''
    Process each row from data and extract structured information about answer choices.
    
    Parameters:
        row : A row of data containing responses to survey questions.

    Returns:
        result_frames : list with information about phone, choice, skill, program, and price.
    '''
    phone = row['Phone']
    result_frames = [] # Empty list for store the result
    
    for i in range(1, 11):  # Iterate through Q1 to Q10
        answer = row[f'Q{i}'].split(', ') # Extract answer from every question
        answer_desc = [] # Empty list to store dictionary about answer description
        for choice, choice_info in options[f'Q{i}'].items(): # Iterates through the answer for the current question 
            choice_value = 1 if choice in answer else 0 # Give value to choice
            answer_desc.append({
                'User_Phone': phone,
                'Choice': choice_value,
                'Skill': choice_info['Skill'],
                'Program': choice_info['Program'],
                'Price': choice_info['Price']
            })
        result_frames.append(pd.DataFrame(answer_desc)) # Appends the answer_desc list to the result_frames list
    return result_frames

# Process each row and concatenate the results
result_frames = filtered_data.apply(process_row, axis=1)
final_result = pd.concat([frame for frames in result_frames for frame in frames], ignore_index=True)

final_result.head(20)

,User_Phone,Choice,Skill,Program,Price
0,08xx336019xxx,0,Create Analytics Dashboard,Tutorial Based,"Rp 500.000,0"
1,08xx336019xxx,1,Perform Customer Segmentation,Mentoring Based,"Rp 350.000,0"
2,08xx336019xxx,0,Design AB Test Experimentation,Mentoring Based,"Rp 300.000,0"
3,08xx336019xxx,1,Create Analytics Dashboard,Tutorial Based,"Rp 500.000,0"
4,08xx336019xxx,0,Design Data Pipeline,Mentoring Based,"Rp 300.000,0"
5,08xx336019xxx,0,Perform Credit Scoring Analytics,Mentoring Based,"Rp 550.000,0"
6,08xx336019xxx,1,Perform Customer Segmentation,Mentoring Based,"Rp 350.000,0"
7,08xx336019xxx,0,Perform Customer Segmentation,Tutorial Based,"Rp 450.000,0"
8,08xx336019xxx,0,Design Data Pipeline,Mentoring Based,"Rp 250.000,0"
9,08xx336019xxx,1,Design AB Test Experimentation,Mentoring Based,"Rp 500.000,0"


In [12]:
# Check total rows (total user*10*3 --> 285*10*3 = 8550)
total_rows = len(final_result)
print("Total rows:", total_rows)

Total rows: 8550


In [13]:
# save to csv format
final_result.to_csv('data_clean.csv', index=False)